# Introduction

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/leonsnill/geeo/blob/master/docs/habitat-type-mapping_geeo.ipynb)

Remote sensing plays an extraordinarily important role in ecological and biogeographic research and monitoring, such as in assessing biodiversity and habitat state and dynamics (Pereira et al. 2013). The spectral imagery recorded from Earth Observation (EO) satellites either directly measures or proxies a variety of biophysical properties that could otherwise not be measured as consistently across space and time. Medium spatial resolution imagery (~10-100m), as provided by the Landsat and Sentinel-2 missions, offers systematically collected, standardized, and quality assessed (predominantly radiometry and geometry) multispectral time series at unprecedented spatial and temporal coverage and density. As such, Landsat and Sentinel-2 allow for deriving a wide range of essential products needed for studying the Biosphere, including—for example—land cover (LC) and LC change, disturbance regimes (e.g., fire), and vegetation condition and phenology (Radeloff et al. 2024). Although global, standardized products are also becoming increasingly available at medium spatial resolution, there is equally great demand for regional products and research-tailored image analyses (Tulbure et al. 2022), particularly when attempting to bridge the gap between field-based ecological monitoring and remote sensing.

Tailoring the image processing to specific research questions requires, to some extent, specialized knowledge in image processing and earth observation methods. When data volumes become large because broad-scale and long- and dense time series are needed, it can create an additional barrier for non-experts in remote sensing. The earth observation community has developed established preprocessing and feature generation routines for these cases, yet their accessibility to applied ecologists and biogeographers could be improved. Simultaneously, access to a standardized workflow would generally enhance efficiency - including for acquainted remote sensing users—particularly for recurring (pre-)processing steps underpinning a variety of applied EO methods.

**The goal of this notebook is to illustrate the capabilities of geeo in supporting geographical and ecological research that makes use of medium resolution multispectral imagery along worked examples. Specifically, we will illustrate how geeo can support the processing and exporting of various data products that have been used in past research.**

### Worked examples
The study area for the worked examples is a 150x150km region covering various ecosystem/biome types from temperate forests, to Mediterranean shrublands, and alpine grasslands at the French-Italian border of the European Alps. We are attempting to generate remote sensing products to characterise different habitat types and assess vegetation state and dynamics. Specifically, we will cover:

1. Retrieving habitat metrics/indices based on multi-temporal Landsat imagery
2. Creating equidistant, gap-free spectral time series for phenological analysis
3. Land cover and habitat type classification workflow: Combining server-side with local analysis

# Setup

For this notebook we are assuming you went through the setup of geeo, i.e. installing the package and its requirements, ensuring access to Google Earth Engine. Otherwise, we recommend you read the GitHub packeg of GEEO and have a look at the first tutorial notebook "Introducing GEEO".

First, we are going to import all necessary packages:

In [ ]:
my_project_name = 'eexnill'  # change this to your Google Cloud project name that has access to Earth Engine

# earth engine python API
import ee
ee.Authenticate()
ee.Initialize(project=my_project_name)
# eerepr: Add interactive Code Editor-style HTML reprs to Earth Engine objects in a Jupyter environment
import eerepr
eerepr.initialize()
# geemap: interactive mapping with Earth Engine in Python
import geemap
import pandas as pd

# GEEO
try:  # if geeo is already installed in environment
    import geeo
except:   # not installed, install it
    !pip install --quiet git+https://github.com/leonsnill/geeo.git
    import geeo

**A remark:** In the following sections we commented-out some of the code or changed the parameter settings in order to not trigger unneccessary exports and keep server requests to a minimum. We still show all results and ran all the processing a priori so you can simply follow along.

# Study Area

Our study area is a 150x150km large area at the French-Italian border covering various ecosystem/biome types from temperate forests, to mediterranean shrublands, and alpine grasslands. We have prepared a .gkpg-file containing the boundaries of our study area. We can translate a local vector file to a server-side ee.Geometry using a build-in helper-function in GEEO named `create_roi()`:

In [ ]:
study_area = 'https://raw.githubusercontent.com/leonsnill/geeo/master/geeo/data/GLANCE_EU_150-X029-Y029.gpkg'
roi = geeo.create_roi(study_area)  # returns a dictionary of various formats, including an ee.Geometry
roi.keys()

`create_roi()` returns a dictionary containing our study area vector in different formats: as [ee.Geometry](https://developers.google.com/earth-engine/guides/geometries) (roi_geom), [ee.FeatureCollection](https://developers.google.com/earth-engine/guides/feature_collections) (roi_featcol), an ee.Geomtry of the bounding coordinates (roi_bbox), and the [GeoDataFrame](https://geopandas.org/en/stable/docs/reference/geodataframe.html#) (roi_gdf; can either be a bounding box or full geometry).

We will use the ee.FeatureCollection and visualize the extent of our study area using geemap:

In [ ]:
roi_featcol = roi['roi_featcol']  # extract the ee.FeatureCollection
roi_featcol

The resulting ee.FeatureCollection has one feature representing the rectangular study area.

In [ ]:
M = geemap.Map(center=[44, 7], zoom=7)
M.add_basemap('HYBRID')
vis_params = {'color': 'red', 'width': 2, 'lineType': 'solid','fillColor': '00000000'}
M.addLayer(roi_featcol.style(**vis_params), {}, 'Study Area')
M

# Example 1: Retrieving habitat metrics based on multi-temporal Landsat imagery

Multispectral imagery is considered a usefull predictor of habitat conditions and dynamics, since vegetation productivity proxies (food) resources availability. 
As such, continuous spectral features derived from optical remote sensing, that in turn measure productivity (e.g. Normalized Difference Vegetation Index (NDVI)), are frequently used in to characterise habitat and serve as inputs for SDMs. Different relationships between ecological process and specral measure have been established to study biodiversity. Fairly popular are the Dynamic Habitat Indices (DHIs) (e.g. Radeloff et al. 2019; Razenkova et al. 2025) that measure vegetation productivity and seasonality from vegetation indices. Similarily, Oeser et al. (2019) characterize different aspects of wildlife habitat - including productivity, phenology, openness, moisture and snow cover - based on multitemporal multispectral Landsat data in order to map habitat suitability for lynx, red deer, and roe deer in the Bohemian forests of Germany and the Czech Republic.

Despite various nomenclature and ecological semantics, these approaches essentially use what the remote sensing community refers to as 'multitemporal spectral metrics' or 'Spectral-Temporal-Metrics' (STMs). **STMs are a commonly applied form of dimensionality reduction in remote sensing, in which pixel-wise statistics across individual bands/features are calculated for a given set of imagery over time.** For example, one could estimate the average productivity within the growing season based on the NDVI by calculating the pixel-wise statistical average of all NDVI values observed within the defined time period. STMs are popular because they collapse potential high dimensional data (time series) into fewer metrics that if properly selected represent similar amount of information variance/conten, while also solving a ubiquitous problem of optical imagery: gaps in the imagery due to quality masking (e.g., clouds). As such, **STMs are fairly robust and spatially continuous features required for gap-free image analyses such as habitat or land cover mapping**.  

**In this first example, we are going to translate the remote sensing methodology of deriving habitat metrics from Landsat to GEEO** which could underpin similar research conducted in our study area. We will partly orientate the image processing workflow along the methodology by Oeser et al. (2019). In their paper, they combine different spectral features that proxy different aspects of land surface or habitat conditions with various intra-annual time windows that represent different phenological stages and thus seasonal ressource availability. In summary, they use all available Landsat imagery between 2011 and 2013 covering their study area and mask out all observations containing clouds, cloud shadows or snow using the the Pixel-Quality Assessment (QA) band. They calculate three main features from the multspectral bands of Landsat: the Tasseled Cap greenness (TCG), brightness (TCB) and wetness (TCW). TCG proxies photosynthetically active vegetation, TCB proxies the soil component and thus certain types of open habitat, and TCW proxies moisture, including soil and vegetation water content. Based on the TCG, TCB, and TCW time series, they derive a set of STMs (median, interquartile-range, etc.) for different seasonal windows (start-of-season DOY 60-151; peak-of-season DOY 152-243, etc.).

**Defining the settings for the habitat metric calculation**
- We want to capture average conditions of last three years: 2022-2024
- We want to differentiate between spring (Mar-May), summer (Jun-Aug) and autumn (Sep-Nov), and winter (Dec-Feb) season
- We will include all available Landsat-8 to Landsat-9 data with <75% cloud cover and mask all non-clear, invalid pixels except for snow
- We will use the same spectral features as Oeser et al., i.e. TCG, TCB, and TCW (since they have the advantage of using the full multispectral spectrum recorded from Landsat)
- We will calculate the following STMs for each season and feature combination: the median, 10% and 90% percentiles, and standard deviation

Based on our defined criteria for deriving the metrics, we can now move on to how to translate these conditions into processing instructions for geeo. 

## Specifying the processing instructions in GEEO
Because it is easier to illustrate the workflow in this notebook using the dictionary approach, we are going to specify the instructions directly here in a code chunk using a python dictionary instead of creating .yml parameter file. Here is the table of all parameters we should set in order to create a Spectral-Temporal-Metric (STM) ee.ImageCollection that matches our definitions from above:

| Parameter                | Type    | Allowed Values / Format         | Description                                      |
|--------------------------|---------|---------------------------------|--------------------------------------------------|
| YEAR_MIN                 | int     | Any year                        | Start year (inclusive) when DATE_* not set. |
| YEAR_MAX                 | int     | Any year                        | End year (inclusive); paired with MONTH/DOY filters when DATE_* absent. |
| MONTH_MIN                | int     | 1-12                            | First month in seasonal window; can exceed MONTH_MAX to wrap (e.g. 11→2). |
| MONTH_MAX                | int     | 1-12                            | Last month in window; wrapping applied when < MONTH_MIN. |
| ROI                      | list / str / GeoDataFrame / ee object | Point [lon, lat]; BBox [xmin, ymin, xmax, ymax]; path to .shp/.gpkg; GeoDataFrame; ee.Geometry / ee.FeatureCollection | Area of Interest; complex polygons can slow filtering and may be bbox-simplified. |
| | | | | 
| SENSORS                  | list    | L9, L8, L7, L5, L4, S2, HLS, HLSL30, HLSS30 | L9/L8/L7/L5/L4 -> LANDSAT Collection 2 Tier 1 Surface Reflectance product ([LANDSAT/LXXX/C02/T1_L2](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC09_C02_T1_L2)); S2 -> Sentinel-2 ([COPERNICUS/S2_SR_HARMONIZED](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR_HARMONIZED)). L9/L8/L7/L5/L4 + S2 can be merged, e.g. [L9, L8, S2]; HLS/HLSL30/HLSS30 -> Harmonized Landsat ([HLSL30](https://developers.google.com/earth-engine/datasets/catalog/NASA_HLS_HLSL30_v002)), Sentinel-2 ([HLSS30](https://developers.google.com/earth-engine/datasets/catalog/NASA_HLS_HLSS30_v002)) product, and combined (HLS) product. |
| MAX_CLOUD                | int     | 0-100                           | Scene-level metadata cloud cover (%) filter before per-pixel masking. Band used for sensors: CLOUD_COVER_LAND (L9/L8/L7/L5/L4), CLOUDY_PIXEL_PERCENTAGE (S2), and CLOUD_COVERAGE (HLS/HLSL30/HLSS30). |
| MASKS_LANDSAT            | list    | See description                 | Apply listed QA masks (cloud, cshadow, snow, fill, dilated, saturated); null disables. |
| MASKS_LANDSAT_CONF       | str     | Medium, High                    | Confidence threshold for Landsat QA masking (Medium = more conservative; favours error of commission, i.e. cloud remnants less likely, but clear pixels may be falsely masked). |
| | | | | 
| FEATURES                 | list    | See description                 | Bands / indices / custom formula outputs kept in pipeline; include DEM or unmixing outputs here for export. |
| | | | | 
| FOLD_YEAR                | bool    | true, false                     | Partition time series into yearly subwindows. |
| FOLD_MONTH               | bool    | true, false                     | Partition into monthly subwindows (Jan..Dec across years). |
| FOLD_CUSTOM              | dict    | See description                 | Additional custom windows via ranges or target±offset lists (e.g. [2020-2021], [265+-30]). |
| | | | | 
| STM                      | list    | min, p5, ..., max, mean, etc.   | Temporal reducers to apply (e.g. min, max, mean, stdDev, p5–p95); null skips. |
| STM_BASE_IMGCOL          | str     | TSS, TSM, TSI, CIC              | Collection supplying time series for statistics. |
| STM_FOLDING              | bool    | true, false                     | Compute metrics within each active fold (year/month/custom). |

Note that we could set a few more settings specific to certain aspects of the processing. For example, we could further change some of the quality masking behaviour etc.. For this example, we keep it to the minimum, but still absolutely sufficient settings for achieving the desired outcome.

To translate our settings for the habitat metric calculation to a GEEO compatible dictionary then looks as follows: 

In [ ]:
habitat_metric_settings = {
    'YEAR_MIN': 2022,
    'YEAR_MAX': 2024,
    'MONTH_MIN': 1,                                 # we include all months at first, the seasonal subwindows are defined in the folding section
    'MONTH_MAX': 12,
    'ROI': study_area,                              # our study area, note we could also supply the path to our .gpkg here
    'SENSORS': ['L8', 'L9'],                        # Landsat-8 and Landsat-9 Collection 2 Tier 1 surface reflectance data
    'MAX_CLOUD': 75,                                # only include scenes with less or equal 75% cloud cover
    'MASKS_LANDSAT': [                              # mask all pixels of this kind
        'cloud', 'cshadow', 
        'fill', 'dilated'
        ],  
    'MASKS_LANDSAT_CONF': 'Medium',                 # lower certainty boundary, Medium masks more pixels to favour comission error over omission 
    'FEATURES': ['TCG', 'TCB', 'TCW'],              # Tasseled Cap greenness, brightness, and wetness
    'FOLD_YEAR': False,                             # we do not wish to calculate metrics for each year separately
    'FOLD_MONTH': False,                            # neither for each month
    'FOLD_CUSTOM': {
        'year': None,
        'month': ['3-5', '6-8', '9-11', '12-2'],    # instead, we want a custom range of months
        'doy': None, 
        'date': None
        },  
    'STM': ['p10', 'p50', 'p90', 'stdDev'],         # percentile 10, 50 (median), 90, and the standard deviation
    'STM_BASE_IMGCOL': 'TSS',                       # collection on which to calculate the STMs; here we use the baseline Time-Series-Stack (TSS)
    'STM_FOLDING': True,                             # actually apply the custom folding settings when calculating the STMs
    'STM_FOLDING_LIST_ITER': False
}

Please be reminded that usually when tweaking many parameters, a more efficient approach - particularly for new users - is to use a parameter file and we only use the dictionary approach for better comprehensibility of this tutorial.

## Running the processing instructions
Once we have the settings in place, we can run the processing using the `run_param()` function. As described in the introduction tutorial, `run_param` is a wrapper function that executes the chain of `run_level2()` -> `run_level3()` -> `run_level4` -> `run_export()`, where each output of each module is fed into the subsequent one. Each module expects a dictionary as input, and also returns a dictionary plus added variables (e.g. processed ee.ImageCollections) as output:

In [ ]:
run_prm = geeo.run_param(habitat_metric_settings)

The `run_prm` variable is a Python dictionary and contains all parameter settings, i.e., the ones we specified + the default ones we did not specify, as well as all specified products we wished to calculate. In our case this is the Time Series Stack (TSS) matching Level-2 processing of our Landsat-8 and Landsat-9 archives, and the Spectral-Temporal-Metrics (STMs). Printing the `run_prm` variable returns many rows of key-value pairs:

In [ ]:
run_prm

The returned dictionary `run_prm` now contains an ee.ImageCollection at the STM key for which we previously defined the metrics to calculate:

In [ ]:
STM = run_prm['STM']  # the calculated metrics are stored in an ee.ImageCollection at the STM key
STM  # print STM ee.ImageCollection object and inspect its properties

Inspect the STM object by uncollapsing the attributes. We see the STM object contains four images, each in turn containing twelve bands. The four images match our four specified subwindows, in turn matching our defined northern hemispheric seasons for our study area. When you uncollaps the first image, for example, you can observe the list of bands containing the specified feature-metric combination (e.g. TCG_p10).

When using STM in combination with temporal folding, we might loose track of which image represents which time period. The `system:index` property contains the temporal information of the image in an easily readable format. For example, the first image displays `Y2022-2024_M03-05_D001-366`.

In essence this is all that is needed to get the desired image results. We could now also have simply requested to export the STMs to Drive or as Asset, but here we are going to assess the result in the interactive python session (this may run for a short time since requesting a map view now triggers the actual processing of the data on the server):

In [ ]:
img_spring = STM.first()  # first image in the collection, which is spring here

M = geemap.Map(center=[44, 7], zoom=7)
M.add_basemap('HYBRID')
M.addLayer(img_spring, {'bands': ['TCB_p50', 'TCG_p50', 'TCW_p50'], 'min': [0.1, 0, -0.2], 'max': [0.6, 0.3, 0]}, 'Spring median STM')
M.addLayer(roi_featcol.style(**vis_params), {}, 'Study Area')
M

## Exporting the image data
Often we would like to export data to Drive or as an Asset. If we are requesting an export from Earth Engine (via geeo), we either directly or indirectly specify metadata associated with the export. For example, an image export requires a Coordinate Reference System (CRS), spatial extent (defined by ROI), and pixel resolution (called 'scale' in GEE) to be set. In plain GEE, the user must at least specify the region and scale (pixel resolution) for the export to work, but this means all other metadata and precise CRS information are inferred. It is generally good practise to be explicit about the metadata and the CRS. 

geeo encourages being explicit and allows full control on specific metadata, for example, the CRS transform and image dimensions (https://developers.google.com/earth-engine/guides/exporting_images). At the same time, geeo can handle precisely aligning CRS grids etc. for the user. That means, by default it will align the output extent (precisely) to the input ROI. Read more on tiling schemes and CRS here.

In our example, we use a 150x150km tile that defines our study area. In this case we know it is projected in a specific CRS following the GLANCE tiling scheme (https://measures-glance.github.io/glance-grids/). geeo knows these continent specific CRS (EU, NA, SA, AF, AS, OC) and we can thus specify "CRS: EU" in our parameter settings. Furthermore, we are working with Landsat data that predominantly records with a 30m spatial resolution which we intend to keep. Since our process involves many Landsat scenes, each stored in potentially different UTM CRSs, and we intend to reproject to the EU GLANCE CRS, we would like to use bilinear resampling as spatial interpolation method for our continuous spectral data (instead of nearest neighbour). To store data more efficiently, we accept the standard settings of scaling our data by a value of 10,000 and saving the resulting scaled data using the signed integer 16 data type (instead of floating point). Invalid, or missing pixels should be set to -9999. Last, we specify that we would like to export image products in general, and the STM in particular. We further specify the location to Asset and the associated directory within the Assets:

In [ ]:
habitat_metric_settings_export = {
    'PIX_RES': 30,  # spatial resolution
    'CRS': 'EU',  # EU: Europe glance (https://measures-glance.github.io/glance-grids/)
    'RESAMPLING_METHOD': 'bilinear',  # bilinear resampling for continuous data
    'DATATYPE': 'int16',  # output data type ...
    'DATATYPE_SCALE': 10000,   # ... and scale factor to convert to integer
    'NODATA_VALUE': -9999,  # output nodata value
    'EXPORT_IMAGE': False,  # whether to export the images: setting to true would trigger the export
    'EXPORT_STM': True,  # whether to export the calculated STMs
    'EXPORT_LOCATION': 'Asset',  # 'Drive' or 'Asset'
    'EXPORT_DIRECTORY': 'projects/eexnill/assets/geeo_public',  # path to export location
    'EXPORT_DESC': 'STM_HABITAT_METRICS_2022-2024_LSAT'  # description appended to each image name
}

habitat_metric_settings.update(habitat_metric_settings_export)

run_prm = geeo.run_param(habitat_metric_settings)  # uncomment to run the export (note that EXPORT_IMAGE is intentionally set to False here because we ran the export for illustrational purposes already)

Once the export is finished, the exported STM image will show up under the specified EXPORT_DIRECTORY and EXPORT_DIRECTORY combination (here: `projects/eexnill/assets/geeo_public/STM_HABITAT_METRICS_2022-2024_LSAT`. We can then access the exported data as follows:

In [ ]:
img_habitat = ee.Image('projects/eexnill/assets/geeo_public/STM_HABITAT_METRICS_2022-2024_LSAT')
print(img_habitat.bandNames().getInfo()[:5])  # print the first 5 band names

Printing the first five bandnames illustrates how we can uniquely identify the features we exported. The temporal folding information is appended to the bandnames (e.g. 'Y2022-2024_M03-05_D001-366'), followed by the feature (e.g. 'TCG') and associated statistical metric/reducer (e.g. 'p10').

We can visualize the exported STM ee.Image using geemap:

In [ ]:
M = geemap.Map(center=[44.5, 7], zoom=8)
M.add_basemap('HYBRID')
selected_bands = ['Y2022-2024_M03-05_D001-366_TCG_p50', 'Y2022-2024_M06-08_D001-366_TCG_p50', 'Y2022-2024_M09-11_D001-366_TCG_p50']
M.addLayer(img_habitat, {'bands': selected_bands, 'min': -60, 'max': 2600}, 'Multitemporal median TCG')
M.addLayer(roi_featcol.style(**vis_params), {}, 'Study Area')
M

Now we could continue with our analysis in Earth Engine using the ee.Image Asset or export it for local analysis to our Google Drive (in case the latter would have our intention, we could have directly specified `EXPORT_LOCATION: Drive` in our parameter file/dictionary instead).

## Summary
With a few lines of instructions provided via the parameter file/dictionary, we were able to adopt the methodology decribed by Oeser et al. (2019) for our study area and time period. The resulting image products can be used as ecological indicators suitable for a range of applications, including habitat modelling.


<br>

----

<br>

# Example 2: Time-Series Interpolation (TSI)

In this section we are going to create a Time Series Interpolation (TSI) from our Landsat Time Series Stack (TSS). With STMs we already learned how to reduce the inconsistent, gappy time-series of different density into a continuous image product suitable, for example, for mapping exercises.

In the second example, we will derive a Time Series Interpolation (TSI) product from a Landsat-based Time Series Stack (TSS) using a double Radial Basis Function (‘Gaussian’) interpolation. Many applications use interpolated spectral time series, including, for example, assessments of land-use intensity (Rufin et al., 2019) or characterisiation of land surface phenology (Kowalski et al., 2020). 

TSIs uses interpolation methods to create a gap-free, equidistant time series which can be usefull for applications that require these attributes, i.e. when gappy, non-equidistant and fluctuating data densities directly impact the analysis. TSI can help suppress undesired variance (or noise) and allows for uninflated calculations of image features such as Spectral-Temporal-Metrics (Nill et al., 2022).

#### A bit of background on RBF Time Series Interpolation
Originally, the spectral time series of Landsat/Sentinel are not equidistant, i.e. the amount of valid pixels varies over time (e.g., due to cloud cover).
To ensure a consistent, gap-free time series, the data can be interpolated into an equidistant time series. 
Consider the following time series plot showing the NDVI for a single pixel between 2010 and 2016:

<div>
<img src="https://raw.githubusercontent.com/leonsnill/geeo/master/geeo/data/fig/NDVI_time_series.png" width="60%"/>
</div>

To ensure a gap-free, equidistant time series which captures the reflectance dynamics smoothly, yet also is robust to potential outliers, a common interpolation method uses a Radial-Basis-Function (RBF) or Gaussian kernel. This interpolation method is a convolution, so no function fitting is applied, but instead the Gaussian function acts as a moving window averaging. The RBF kernel 'moves' over the desired equidistant time steps, and the original observations (e.g. Time Series Stack (TSS)) are weighted according to a Gaussian distribution. 
If we used a single RBF kernel and `sigma=16` (days) for the width parameter of the Gaussian function, this is how a smoothed time series looked like:

<div>
<img src="https://raw.githubusercontent.com/leonsnill/geeo/master/geeo/data/fig/NDVI_time_series_1RBF.png" width="60%"/>
</div>

You can see this is of not much help for data scarce periods and also mimics the data
points too closely.
Usually, data density is variable, which means that we have seasons with
good data availability and seasons with less ideal data availability.
Where there are many observations, we would want to have a narrow kernel to more closely
follow the actual time series. For data poor situations, a narrow kernel would
result in many nodata values and would not smooth the time series appropriately.
As a solution, we use multiple RBF functions with different widths and weight the
influence of each RBF kernel based on the actual data availability.
Using two kernels, one 16 day, one 32 day, we get the following result:

<div>
<img src="https://raw.githubusercontent.com/leonsnill/geeo/master/geeo/data/fig/NDVI_time_series_2RBF.png" width="60%"/>
</div>

Of course, do not expect brilliant results if input data availability is low. Also keep in mind this is an interpolation and it creates an artificial image product. Smooth-looking results does not mean smooth data quality!

Here is the table of all parameters we can set in order to create a TSI in geeo:

| Parameter                | Type    | Default      | Allowed Values / Format         | Description                                      |
|--------------------------|---------|--------------|---------------------------------|--------------------------------------------------|
| TSI                      | str     | null         | null, WLIN, 1RBF, 2RBF, 3RBF    | Interpolation method: WLIN (weighted linear) or Radial-Basis-Function (RBF) using one to three Gaussian kernels; null disables. |
| TSI_BASE_IMGCOL          | str     | TSS          | TSS, TSM, CIC                   | Collection to interpolate (raw, mosaicked, or custom). |
| INTERVAL                 | int     | 16           | days                            | Desired spacing between interpolated timestamps. |
| INTERVAL_UNIT            | str     | day          | day, month, year                | Unit used for INTERVAL spacing. |
| INIT_JANUARY_1ST         | bool    | false        | true, false                     | Force interpolation to start Jan 1 (of YEAR_MIN). |
| SIGMA1                   | int     | 16           | days                            | First RBF kernel width (days). |
| SIGMA2                   | int     | 32           | days                            | Second RBF kernel width (2RBF/3RBF only). |
| SIGMA3                   | int     | 64           | days                            | Third RBF kernel width (3RBF only). |
| WIN1                     | int     | 16           | days                            | +- of days (i.e. half-window size) maximum offset for observations in TSI_BASE_IMGCOL to be included for each INTERVAL for kernel 1.  |
| WIN2                     | int     | 32           | days                            | +- of days (i.e. half-window size) maximum offset for observations in TSI_BASE_IMGCOL to be included for each INTERVAL for kernel 2. |
| WIN3                     | int     | 64           | days                            | +- of days (i.e. half-window size) maximum offset for observations in TSI_BASE_IMGCOL to be included for each INTERVAL for kernel 3. |
| BW1                      | int     | 4            |                                 | Expected/ideal number of observations between two timesteps (e.g. within 16 days when INTERVAL=16) used to calculate the weight for 2nd kernel (2RBF/3RBF) over the first. If there are enough observations to satisfy the ideal setting for a single kernel, the second kernel weight is set to zero, i.e. obervations further away do not impact the interpolation result. |
| BW2                      | int     | 8            |                                 | Weight for 3rd kernel (3RBF), see BW1. |

When attempting to use RBF interpolation, the user can specify `1RBF`, `2RBF`, or `3RBF` to either use a single, two, or three independent Gaussian functions to interpolate the time series. Note that more kernels is not necessarily better while definitely being more expensive computationally.

### TSI for phenological analysis
We will stay within our study area and time period and would like to create a TSI for our Tasseled Cap features and the Normalized Difference Vegetation Index (NDVI). This time, we will use a parameter file for this exercise (geeo/docs/landsat-tsi-habitat.yml). We already adjusted the settings in the .yml file and we can load the parameters in this interactive session like so:

In [ ]:
# load parameters from the .yml file into a dictionary
prm_tsi = geeo.load_parameters('https://raw.githubusercontent.com/leonsnill/geeo/master/docs/landsat-tsi-habitat.yml')

# we will print the parameters and associated values for illustration
new_parameters_of_interest = [
    'TSI',
    'TSI_BASE_IMGCOL',
    'INTERVAL',
    'INTERVAL_UNIT',
    'INIT_JANUARY_1ST',
    'SIGMA1',
    'SIGMA2',
    'WIN1',
    'WIN2',
    'BW1'
]

for p in new_parameters_of_interest:
    print(f"{p}: {prm_tsi[p]}")

We can see that we wish to use two RBF kernels (`TSI: 2RBF`) based on our original (preprocessed) Landsat TSS (`TSI_BASE_IMGCOL: TSS`). We further would like to have a equidistant interval of 16 days (`INTERVAL: 16` and `INTERVAL_UNIT: day`), starting off January 1st (`INIT_JANUARY_1ST: True`) 2022 (the latter is defined by the `YEAR_MIN` parameter). For our two gaussian kernels, we set the width (sigma) of the bell curve to 16 and 32 days respectively (`SIGMA1: 16` and `SIGMA2: 32`), and would like to only include observations falling within +-30 and +-90 days of our target time steps (`WIN1: 30` and `WIN2: 90`), i.e., any observation exceeding these windows will not influence the interpolated value. `BW1: 4` means that if there more or equal to four original, valid observations within the first window (`WIN1`) for a given interpolation time step, the second kernel will not influence the interpolated value (because sufficient observations are available for the first kernel to be fully active).

With these settings in place, we run the image export (using the export settings for pixel size, CRS, etc. we already used for the STM example):
(Note that we set EXPORT_IMAGE to False again so that you do not trigger the export by running the notebook)

In [ ]:
my_run = geeo.run_param(prm_tsi)  # runs the analysis as specified in the .yml file

After the exports are finished, we can continue with our analysis. In our case, we have exported the images as Asset and they can be accessed like so:

In [ ]:
tsi_tcg = ee.Image('projects/eexnill/assets/geeo_public/landsat-tsi/TSI_HABITAT_EU_150-X029-Y029_2022-2024_LSAT_TCG')
tsi_tcb = ee.Image('projects/eexnill/assets/geeo_public/landsat-tsi/TSI_HABITAT_EU_150-X029-Y029_2022-2024_LSAT_TCB')
tsi_tcw = ee.Image('projects/eexnill/assets/geeo_public/landsat-tsi/TSI_HABITAT_EU_150-X029-Y029_2022-2024_LSAT_TCW')
tsi_ndvi = ee.Image('projects/eexnill/assets/geeo_public/landsat-tsi/TSI_HABITAT_EU_150-X029-Y029_2022-2024_LSAT_NDVI')

Checking the bandnames of the TSI NDVI image:

In [ ]:
tsi_ndvi.bandNames()

Visualizing the interpolated NDVI time series for three distinct moments in time ('2024-05-14', '2024-07-01', '2024-09-19'):

In [ ]:
# visualize TSI
example_point = ee.Geometry.Point([5.6668306, 44.5112991])

M = geemap.Map(center=[44.5112991, 5.6668306], zoom=8)
M.add_basemap('HYBRID')
selected_bands = ['NDVI_20240514', 'NDVI_20240701', 'NDVI_20240919']
M.addLayer(tsi_ndvi, {'bands': selected_bands, 'min': 1000, 'max': 9000}, 'TSI NDVI')
M.addLayer(example_point, {'color': 'red'}, 'Example Point')
M.addLayer(roi_featcol.style(**vis_params), {}, 'Study Area')
M

Visualizing the TSS and TSI of the NDVI for the example point:

In [ ]:
from geeo import plot_getRegion
import matplotlib.pyplot as plt

TSS = my_run['TSS']
TSI = my_run['TSI']
#fig, ax = plt.subplots(figsize=(10, 4))
#plot_getRegion(TSS, band='NDVI', roi=example_point, scale=30, axis=ax, style='x')
#plot_getRegion(TSI, band='NDVI', roi=example_point, scale=30, axis=ax, style='.-', color='blue')

<div>
<img src="https://raw.githubusercontent.com/leonsnill/geeo/master/geeo/data/fig/TSI_example_NDVI.png" width="50%"/>
</div>

This is how you can create an equidistant, gap-free (if orignal data allows) time series. As you can see, some observations that could be considered outliers are also smoothed-out in the resulting TSI and data dense intervals are reduced to the average conditions and represented in much fewer observations. 

<br>

----

<br>

# Example 3: European-wide land cover mapping

The third example showcases how GEEO can be incorporated into a larger workflow. Specifically, we illustrate this in the context of large-area land cover mapping analogously to the methodology described by Pflugmacher et al., (2019). In this paper, the authors use the LUCAS 'Land use and land cover' survey reference dataset in combination with multispectral Landsat-8 data to map pan-European land cover for the year 2015.

Here, we retrieved Landsat Level-2 Time Series Stack (TSS) data for thousands of LUCAS reference points across Europe. We then used the data to derive a large variety of Spectral-Temporal-Metrics (STMs) which in turn served as features in Random Forest classification modelling. We then uploaded the final model to Earth Engine which allows for predicting twelve land-cover classes. We describe the overarching steps and provide links to the Python files implementing the methods, but focus on the main outputs and results in this notebook.

## LUCAS - Land use and land cover survey
The Land Use/Cover Area frame Survey (LUCAS) is a harmonised, in situ survey conducted across the entire European Union (EU) to provide comparable statistics on land use and land cover. LUCAS collects field-based observations using a standardised methodology, enabling the study of socio-environmental challenges such as land take, soil degradation, and biodiversity loss. Land cover describes the physical characteristics of the land (e.g., forests, water, buildings), while land use refers to how land is utilised (e.g., agriculture, residential, recreation). Land cover is classified in a total of 8 categories, 29 classes and 76 subclasses. Pflugmacher et al. (2019) differentiate between 12 land cover classes:

| Main Class             | Short Description                                                                                   |
|------------------------|----------------------------------------------------------------------------------------------------|
| Artificial land        | Areas with artificial, often impervious cover (buildings, pavement, parking lots, yards).          |
| Cropland, seasonal     | Areas with seasonal crops (cereals, root crops, vegetables, fodder), excluding temporary grassland.|
| Cropland, perennial    | Areas with permanent crops such as fruit trees and other perennial plantations.                    |
| Forest, broadleaved    | Woodland with >75% broadleaved trees and canopy cover ≥10%.                                       |
| Forest, coniferous     | Woodland with >75% coniferous trees and canopy cover ≥10%.                                        |
| Forest, mixed          | Woodland with both broadleaved and coniferous trees (>25% each), canopy cover ≥10%.               |
| Shrubland              | Areas dominated by shrubs and low woody plants, may include sparse trees (<10% canopy).           |
| Grassland              | Land covered mainly by grasses/forbs, includes permanent pasture, sparse trees/shrubs allowed.     |
| Barren                 | Bare land or areas with lichens/moss, minimal or no vegetation cover.                             |
| Wetland                | Inland or coastal wetlands, including fresh, salty, or brackish water areas.                      |
| Water                  | Inland or coastal water bodies and flooded surfaces, mostly without vegetation.                    |
| Snow/ice               | Areas covered by glaciers or permanent snow.                                                       |

A harmonized version of LUCAS (d'Andrimont et al., 2020) covering field surveys from 2006 to 2018 is [available in Earth Engine](https://developers.google.com/earth-engine/datasets/catalog/JRC_LUCAS_HARMO_THLOC_V1#description). **We processed this ee.FeatureCollection to match the pre-processing and categorization described in Pflugmacher et al. (2019) and uploaded the resulting ee.FeatureCollection to a publicly accessible Asset (`projects/eexnill/assets/geeo_public/LUCAS_HARMO_V1_EO_LC`).** The script to (theoretically) reproduce the dataset is provided as separate Python file (`geeo/docs/habitat-type-mapping/LUCAS-preprocess.py`)

Loading the ee.FeatureCollection reference data:

In [ ]:
lucas = ee.FeatureCollection('projects/eexnill/assets/geeo_public/LUCAS_HARMO_V1_EO_LC')
lucas.size()

Visualizing the 505,509 points ...

In [ ]:
# names for legend
lc_names = [
    "Artificial land",
    "Cropland, seasonal",
    "Cropland, perennial",
    "Forest, broadleaved",
    "Forest, coniferous",
    "Forest, mixed",
    "Shrubland",
    "Grassland",
    "Barren",
    "Wetland",
    "Water",
    "Snow/ice"
]
# colours for classes
lc_colors = [
    "FF0000",  # 0 - Artificial land (red)
    "FFD700",  # 1 - Cropland, seasonal (golden/yellow)
    "CD853F",  # 2 - Cropland, perennial (tan/brown)
    "32CD32",  # 3 - Forest, broadleaved (forest green)
    "006400",  # 4 - Forest, coniferous (dark green)
    "228B22",  # 5 - Forest, mixed (lime green)
    "FFA500",  # 6 - Shrubland (orange/kaki)
    "9ACD32",  # 7 - Grassland (yellow green)
    "D2B48C",  # 8 - Barren (tan)
    "800080",  # 9 - Wetland (purple)
    "0000FF",  # 10 - Water (blue)
    "FFFFFF"   # 11 - Snow/ice (white)
]

styled_lucas = geemap.ee_vector_style(
    lucas, 
    column="LC_ID", 
    labels=list(range(len(lc_names))), 
    color=lc_colors,
    pointSize=1
)

M = geemap.Map(center=[44, 7], zoom=7)
M.add_basemap('HYBRID')
M.addLayer(styled_lucas, {}, "lc1")
M.add_legend(title="Land Cover Classes", keys=[str(l) for l in lc_names], colors=lc_colors)
M

These are the reference points we used to build Random Forest classification models. But first, we exported the Landsat-7/-8/-9 Level-2 analysis-ready Time Series Stack (TSS) data for 2014 to 2019 (covering the 2015 and 2019 LUCAS survey) using GEEO. Additional to the TSS data, we also exported climate (CHELSA) and topographic (Copernicus DEM) variables. The scripts to run the export and merge the exported tables are also provided as supplementary Python files (`geeo/docs/habitat-type-mapping/LUCAS_2_export-TSS.py` and `geeo/docs/habitat-type-mapping/LUCAS_3_merge-TSS.py`).

The resulting ee.FeatureCollection tables extend the above points containing the target land cover classes to also contain the TSS, climate, and topographic features:

In [ ]:
lucas_tss = ee.FeatureCollection('projects/eexnill/assets/geeo_public/landsat-tss/TSS_LUCAS_HARMO_V1_EO_LC_EU')
lucas_tss.first()

In [ ]:
lucas_aux = ee.FeatureCollection('projects/eexnill/assets/geeo_public/landsat-tss/LUCAS_HARMO_V1_EO_LC_CHELSA_DEM')
lucas_aux.first()

## Local analyses
**Once we exported the TSS and auxiliary features we continued locally by creating a large set of Spectral-Temporal-Metrics (STMs) from the TSS data**. We could have exported the STMs directly using GEEO, but exporting the 'pure' analysis-ready-data has the advantage of (a) being more efficient server-side and (b) allowing many applications with the data in the future because we can always derive higher-level products (such as STMs) from the TSS table data.
The resulting dataframe looks like this:

In [ ]:
df_stm = pd.read_csv('https://raw.githubusercontent.com/leonsnill/geeo/master/docs/habitat-type-mapping/STM_LUCAS_HARMO_V1_EO_LC_EU_final_head10.csv')
df_stm

The entire dataframe used for modelling is [available on Google Drive](https://drive.google.com/file/d/11mfBRirCmQYPksie6bQ6zZ6lk8-2bT9_/view?usp=drive_link) (just for completeness but not required for subsequent tutorial!).

Using the STM dataframe, **we then built multiple Random Forest classification models to create a generalized model capable of mapping pan-European land cover.** The scripts to create the STMs and for building the Random Forest models are also provided as supplementary Python files (`geeo/docs/habitat-type-mapping/LUCAS_4_TSS-to-STM.py` and `geeo/docs/habitat-type-mapping/LUCAS_5_RF-model.py`).


## Server-side mapping
The resulting Random Forest model is uploaded in string format as ee.FeatureCollection and can be converted to a ee.Classifier using GEEO:

In [ ]:
rf_model_asset = 'projects/eexnill/assets/geeo_public/RF-MODEL_LUCAS_NTREE-50' 
# geeo has a function to load the table asset to a model
from geeo.level4.model import dt_table_asset_to_model
rf_model = dt_table_asset_to_model(rf_model_asset)

Initially we calculated a large set of STMs and also used all CHELSA BioClim and topographic variables (see STM dataframe above). The above model is the **final model that only requires a subset of features that proved most informative for land cover mapping**. From this feature selection process we obtained the following features:

In [ ]:
selected_features = [
    # full season STMs
    'full_NIR_p25',
    'full_NBR_p5',
    'full_NIR_p95',
    'full_NBR_stdDev',
    'full_RED_p75',
    'full_NBR_p75',
    'full_NBR_p95',
    'full_NDVI_p25',
    'full_SW2_stdDev',
    'full_MDWI_p75',
    'full_NIR_stdDev',

    # sub-season STMs
    'spring_MDWI_p50',
    'spring_SW1_p95',
    'spring_NDVI_p5',
    'spring_NDVI_p75',
    'spring_MDWI_p25',
    'spring_NIR_p95',

    'summer_SW1_p95',
    'summer_NIR_p95',
    'summer_NBR_p5',
    'summer_MDWI_p95',
    'summer_NBR_stdDev',
    'summer_RED_stdDev',

    'autumn_MDWI_p75',
    'autumn_NBR_p25',
    'autumn_SW1_p95',
    'autumn_NDVI_p25',

    # aux variables
    'slope',
    'DEM',
    'bio9',
    'bio6',
    'bio18',
    'bio5'
]

As such, we also only require the above selected features to be calculated as ee.Image when attempting to map land cover server-side on Earth Engine. This saves a lot of ressources and time. In the following we setup the code **to create a predictor image that can serve as input for prediction**.

#### Full season (Jan-Dec) STMs

In [ ]:
# we create a tiling scheme for a new study area of size 600km x 600km
glance_600 = geeo.create_glance_tiles(continent_code='EU', tile_size=600000, land_mask=True)
tile = 'EU_600-X007-Y007'
roi = glance_600[glance_600['ID600'] == tile]

# --------------------------------------------------------------------------
# FULL SEASON (Jan-Dec)
# we setup the parameters to calculate the STMs for our new study area for 2018
prm_lc_mapping = {
    'YEAR_MIN': 2017,
    'YEAR_MAX': 2019,
    'ROI': roi,
    'SENSORS': ['L7', 'L8', 'L9'],
    'MASKS_LANDSAT': ['cloud', 'cshadow', 'fill', 'dilated'],
    'FEATURES': ['RED', 'NIR', 'SW1', 'SW2', 'NDVI', 'NBR', 'MDWI'],
    'FOLD_CUSTOM': {'month': ['3-5', '6-8', '9-11']},  # spring, summer, autumn
    'STM': ['p5', 'p25', 'p50', 'p75', 'p95', 'stdDev'],
    'STM_FOLDING': False,
    'RESAMPLING_METHOD': 'bilinear',
}

# run
full_run_lc = geeo.run_param(prm_lc_mapping)
# get STM and rename bands to match RF model
full_STM = full_run_lc['STM'] 
initial_bandnames = full_STM.bandNames().getInfo()  # get band names
new_bandnames = [f'full_{b}' if not b.startswith('full_') else b for b in initial_bandnames]  # adjust to match the RF model
full_STM = full_STM.multiply(10000).rename(new_bandnames)  # rescale and rename bands


#### Sub-season STMs

In [ ]:
prm_lc_mapping.update({'STM_FOLDING': True})  # now set folding to True to get seasonal metrics

# run
seasonal_run_lc = geeo.run_param(prm_lc_mapping)
seasonal_STM = seasonal_run_lc['STM']  # get STM

# rename bands to match RF model
season_prefixes = ['spring', 'summer', 'autumn']

def set_season_index(img, idx):
    return img.set('system:index', season_prefixes[idx])

images = seasonal_STM.toList(seasonal_STM.size())
seasonal_STM = ee.ImageCollection([
    set_season_index(ee.Image(images.get(i)), i)
    for i in range(3)
])

seasonal_STM = seasonal_STM.toBands().multiply(10000)  # scale to match int16 stored in model

#### Auxiliary features: DEM + CHELSA

In [ ]:
dem = ee.ImageCollection('COPERNICUS/DEM/GLO30').filterBounds(full_run_lc['ROI_GEOM']).select('DEM')
dem = ee.Image(dem.mosaic().set('system:time_start', dem.first().get('system:time_start'))).setDefaultProjection(ee.Projection('EPSG:3857'), scale=30)
dem = dem.addBands(ee.Terrain.slope(dem).rename('slope'))
chelsa = ee.Image('projects/eeleeon/assets/CHELSA/CHELSA_BIOCLIM_1981-2010_V21')

#### Predict
Now that we have all predictors in place, we can combine them into one ee.Image that can serve as input to the model prediction:

In [ ]:
img = ee.Image(full_STM.addBands(seasonal_STM).addBands(dem).addBands(chelsa)).select(selected_features)

# predict land cover
land_cover = img.classify(rf_model)
land_cover = land_cover.toUint8()  # cast to uint8 to save space

Export the land cover prediction as image to Asset:

In [ ]:
'''
my_export = geeo.export_img(img=land_cover,
                            px_res=30, region=roi, crs='EU', 
                            out_location='Asset', out_dir='projects/eexnill/assets/geeo_public', outname='LUCAS_LC_2022-2024_RF_EU_600-X007-Y007',
                            dtype='uint8', nodata=255, pyramidingPolicy={'.default': 'mode'})
'''

In [ ]:
cl_img = ee.Image('projects/eexnill/assets/geeo_public/LUCAS_LC_2022-2024_RF_EU_600-X007-Y007')

vis_params_cl = {
    'min': 0,
    'max': 11,
    'palette': lc_colors,
    'bands': ['classification']
}
vis_params = {'color': 'red', 'width': 2, 'lineType': 'solid','fillColor': '00000000'}

M = geemap.Map(center=[45, 7], zoom=6)
M.add_basemap('HYBRID')
M.addLayer(cl_img, vis_params_cl, 'Classification')
M

# References

d’Andrimont, R. et al. (2020) ‘Harmonised LUCAS in-situ land cover and use database for field surveys from 2006 to 2018 in the European Union’, Scientific Data, 7(1), p. 352. Available at: https://doi.org/10.1038/s41597-020-00675-z.

Kowalski, K. et al. (2020) ‘Characterizing spring phenology of temperate broadleaf forests using Landsat and Sentinel-2 time series’, International Journal of Applied Earth Observation and Geoinformation, 92, p. 102172. Available at: https://doi.org/10.1016/j.jag.2020.102172.

Nill, L. et al. (2022) ‘Arctic shrub expansion revealed by Landsat-derived multitemporal vegetation cover fractions in the Western Canadian Arctic’, Remote Sensing of Environment, 281, p. 113228. Available at: https://doi.org/10.1016/j.rse.2022.113228.

Oeser, J. et al. (2019) ‘Habitat metrics based on multi‐temporal Landsat imagery for mapping large mammal habitat’, Remote Sensing in Ecology and Conservation, pp. 1–18. Available at: https://doi.org/10.1002/rse2.122.

Pereira et al. (2013): Essential Biodiversity Variables. Science 339, 277–278. https://doi.org/10.1126/science.1229931

Pflugmacher, D. et al. (2019) ‘Mapping pan-European land cover using Landsat spectral-temporal metrics and the European LUCAS survey’, Remote Sensing of Environment, 221(December 2018), pp. 583–595. Available at: https://doi.org/10.1016/j.rse.2018.12.001.

Radeloff, V.C. et al. (2019) ‘The Dynamic Habitat Indices (DHIs) from MODIS and global biodiversity’, Remote Sensing of Environment, 222, pp. 204–214. Available at: https://doi.org/10.1016/j.rse.2018.12.009.

Radeloff et al. (2024): Need and vision for global medium-resolution Landsat and Sentinel-2 data products. Remote Sens. Environ. 300, 113918. https://doi.org/10.1016/j.rse.2023.113918

Razenkova, E. et al. (2025) ‘Medium-resolution Dynamic Habitat Indices from Landsat and Sentinel-2 satellite imagery’, Ecological Indicators, 173, p. 113367. Available at: https://doi.org/10.1016/j.ecolind.2025.113367.

Rufin, P. et al. (2019) ‘Mapping Cropping Practices on a National Scale Using Intra-Annual Landsat Time Series Binning’, Remote Sensing, 11(3), p. 232. Available at: https://doi.org/10.3390/rs11030232.

Tulbure et al. (2022): Regional matters: On the usefulness of regional land-cover datasets in times of global change. Remote Sens. Ecol. Conserv. 8, 272–283. https://doi.org/10.1002/rse2.248

---